In [28]:
import urllib.request
url = "https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407"
response = urllib.request.urlopen(url)
print (response.read())


b'\n<div id="verfolgen-main-content">\n  \n\n\n\n\n\n\t\n\n\n<script>\n  \n    window.__INITIAL_APP_STATE__ = {\n      initialState: JSON.parse("{\\"sendungen\\":[{\\"id\\":\\"00340434188193324407\\",\\"hasCompleteDetails\\":true,\\"sendungsinfo\\":{\\"gesuchteSendungsnummer\\":\\"00340434188193324407\\",\\"sendungsrichtung\\":\\"ANKOMMEND\\"},\\"sendungsdetails\\":{\\"sendungsnummern\\":{\\"sendungsnummer\\":\\"00340434188193324407\\"},\\"eigenschaften\\":{},\\"sendungsverlauf\\":{\\"datumAktuellerStatus\\":\\"2019-10-10T15:05:00+02:00\\",\\"aktuellerStatus\\":\\"Die Sendung wurde erfolgreich zugestellt.\\",\\"kurzStatus\\":\\"Zustellung erfolgreich\\",\\"fortschritt\\":5,\\"farbe\\":0,\\"events\\":[{\\"datum\\":\\"2019-10-07T16:27:00+02:00\\",\\"status\\":\\"Die Sendung wurde elektronisch angek\\u00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen.\\"},{\\"datum\\":\\"2019-10-08T17:12:00+02:00\\",\\"ort\\":\\"Bremen\\",\\"status\\":\\"Die Sendun

In [32]:
import urllib
import requests
from lxml import html
from lxml import etree
import json


page = requests.get('https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407')
tree = etree.HTML(page.content)

events = tree.xpath('//div')
print(events)




[<Element div at 0x107da2ac8>, <Element div at 0x107da2d48>, <Element div at 0x107da2b48>, <Element div at 0x107da2f48>, <Element div at 0x107da2fc8>]


In [30]:
import json

string = str(events[0])
start = string.find('JSON.parse(')
end = string.find('"),', start)
json_string = string[start:end]
json_string = json_string.replace('JSON.parse("', '')
json_string = json_string.replace('\\', '')
shipment_details = json.loads(json_string)
print(shipment_details)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)